# Dummy Matching Algorithm

In [1]:
from utility_algo import *
import pandas as pd
import numpy as np
from itertools import combinations

In [2]:
data = pd.read_csv('data/sim_data_120_22Apr2020.csv')

data = data.replace('\n','', regex=True)

data = data.reset_index()

data.head()

,index,name,school,subject,grad_yr,zip,industry,title,country,language,connected,priority_tags
0,0,青山 さゆり,['South China Normal University'],Biology,[2020],94102,Banking,Student,France,['Spanish' 'Korean'],[112 119 20 98 31 93],"['tag_1', 'tag_12']"
1,1,傅红梅,['University of California Berkeley'],Mechanical Engineering,[2014],94103,Banking,UX Designer,United States,['Spanish' 'Cantonese'],[118 38 112 11],"['tag_18', 'tag_5', 'tag_15']"
2,2,李红霞,['University of Chicago' 'University of Chicago'],Graphic Design,[2018 2014],94108,Management Consulting,UX Designer,China,['Spanish'],[],[]
3,3,Daniel Wagner,['University of Hong Kong'],Law,[2012],94104,Computer Software,Product Manager,United States,['Chinese' 'Korean' 'Cantonese'],[104 46 17 20 12 20 7 45 103 109 6 22],"['tag_1', 'tag_7']"
4,4,Cristian Santos,['University of Michigan Ann Arbor'],Business Administration and Management,[2020],94101,Financial Services,Product Manager,China,['Korean' 'Japanese'],[107 95 74 85 21 106 88 21 91 28 12 ...,['tag_16']


In [3]:
ii_input = [
#    ('grad_yr', 'MULT'),
    
    # career
    ('industry', 'SINGLE'),
    ('title', 'SINGLE'),
    
    # background
    ('school', 'MULT'),    
    ('subject', 'SINGLE'),
    ('zip', 'SINGLE'),
    ('country', 'SINGLE'),
    ('language', 'MULT'),
    
    # priority
    ('priority_tags', 'MULT')
]

# Form subgroups

The whole pool (all users) based on their two field pair, form subgroups of 2/3/4 ppl

In [4]:
# # helper functions, loaded to .py

# def create_two_category_pair(priority_key='priority_tags'):
#     """
#     use a fixed key and with another key in ii_input to create list of list
#     Think of this as intersection of two sets where one set is fixed
#     eg: this is the combination of priority_tags:tag_18 and language:Korean
#         [['priority_tags', 'language'], ['tag_18', 'Korean'], 3, {16, 17, 53}]
    
#     """
#     remain_data = combined_inverted_index(data.iloc[data.index.isin(list(set(data['index']) - ids_in_subgroups))], ii_input)
#     return two_category_pair(remain_data, priority_key)

# def is_connected_before(data, valid_ids={0, 1, 8, 10}):
#     """
#     When form the subgroups, we don't want to group those connected (or met)
#     If the subgroup has only two ppl, we require they haven't met before
#     If the subgroup has more than 2 ppl, we require they know one at most
#     """
    
#     # if subgroup is two ppl
#     if len(valid_ids) == 2:
#         threshold = 0
#     # if subgroup is more than two ppl
#     else:
#         threshold = 1
#     # Pandas shit, it changed the data structure, anyway what this does is:
#     #    for each uid in valid_ids, use their connected field to check again subgroups
#     #    eg: [0, 0, 1, 2] <- user 3 knows one ppl in the group, user 4 knows two ppl in the group
#     tmp = [len(valid_ids.intersection(set([int(item) for item in data.iloc[i].connected[1:-1].split(' ') if item != '']))) for i in valid_ids]
    
#     return max(tmp) > threshold

# def break_to_subgroups(num=19):
#     """
#     Break the numbers to be consisted of 2 or 3 or 4 only
#     """
    
#     result = list()
#     if num == 4:
#         return [4]
#     elif num == 5:
#         return [2, 3]
#     elif num == 6:
#         return [3, 3]

#     else:
#         result.append([3])
#         result.append(break_to_subgroups(num-3))
#     return [item for sublist in result for item in sublist]

# def remain_users(data, ids_processed, uid_field_name='index'):
#     """
#     based on current processed uid, get the pandas for the remaining users
#     """
#     return data.loc[~data['index'].isin(ids_processed)]


In [5]:
ids_in_subgroups = set()
subgroups = list()

In [6]:
# priority_key means that key must be included as one of the two columns

# order also matters here
priority_keys = ['priority_tags', 'industry', 'school', 'country', 'title', 'language']

print(f'priority order: {priority_keys}\n')

print(f'------start-----to-----form-----subgroup------\n')

# prefer bigger subgroups first, if can't form, then allow for two ppl subgroups
for subgroup_size in [[2, 3, 4], [2, 3, 4]]:
    for priority_key in priority_keys:

        # created intersection set based on two categories
        tmp = create_two_category_pair(data, ids_in_subgroups, ii_input, priority_key)

        # introduce some randomness factors, can assign weight later
        for idx in np.random.choice(range(len(tmp)), len(tmp)//2, replace=False):

            selected_combo = tmp[int(idx)]

            # check who are not in subgroups [need to form subgroups for them]
            valid_ids = tmp[int(idx)][3] - ids_in_subgroups

            # form subgroup in 2/3/4 ppl, also requires most of them haven't met before
            if len(valid_ids) in subgroup_size and not is_connected_before(data, valid_ids):
                ids_in_subgroups.update(valid_ids)
                subgroups.append([selected_combo[0], selected_combo[1], len(valid_ids), valid_ids])
                print(f'subgroup of {len(valid_ids)} ppl is formed for {valid_ids} based on {selected_combo[0]}')

            # if many ppl, break them down into smaller subgroup
            elif len(valid_ids) > 4:
                valid_ids_lst = list(valid_ids)
                # use break_to_subgroups function to decide the break ratio
                for num in break_to_subgroups(num=len(valid_ids_lst)):

                    # haven't connected before, create subgroup
                    if not is_connected_before(data, valid_ids):
                        ids_in_subgroups.update(set(valid_ids_lst[:num]))
                        subgroups.append([selected_combo[0], selected_combo[1], num, set(valid_ids_lst[:num])])
                        print(f'subgroup of {num} ppl is formed for {set(valid_ids_lst[:num])} based on {selected_combo[0]}')                    
                        valid_ids_lst = valid_ids_lst[num:]

                    # connected before, skip this subgroup                    
                    else:
                        valid_ids_lst = valid_ids_lst[num:]

print(f'\n{len(remain_users(data, ids_in_subgroups))} ppl are NOT in the group yet')

priority order: ['priority_tags', 'industry', 'school', 'country', 'title', 'language']

------start-----to-----form-----subgroup------

subgroup of 3 ppl is formed for {64, 107, 4} based on ['priority_tags', 'school']
subgroup of 2 ppl is formed for {112, 113} based on ['priority_tags', 'school']
subgroup of 3 ppl is formed for {116, 62, 55} based on ['priority_tags', 'school']
subgroup of 3 ppl is formed for {16, 67, 53} based on ['priority_tags', 'country']
subgroup of 2 ppl is formed for {56, 51} based on ['priority_tags', 'country']
subgroup of 2 ppl is formed for {81, 49} based on ['priority_tags', 'country']
subgroup of 4 ppl is formed for {75, 23, 13, 31} based on ['priority_tags', 'language']
subgroup of 3 ppl is formed for {89, 115, 44} based on ['priority_tags', 'school']
subgroup of 4 ppl is formed for {11, 59, 3, 39} based on ['priority_tags', 'zip']
subgroup of 2 ppl is formed for {32, 76} based on ['priority_tags', 'country']
subgroup of 3 ppl is formed for {50, 60, 119}

In [7]:
len(ids_in_subgroups)

117

In [8]:
remain_users(data, ids_in_subgroups)

,index,name,school,subject,grad_yr,zip,industry,title,country,language,connected,priority_tags
22,22,Tommy Walter,['University of Southern California'],Economics,[2020],94101,Accounting,Researcher,China,['Korean'],[ 77 74 102 8 89 15 68 115],[]
26,26,佐々木 くみ子,['The Chinese University of Hong Kong' 'Univer...,Computer Science,[2017 2010],94110,Banking,Data Scientist,France,['Cantonese'],[ 89 3 102 38 113 34 0 23 56],[]
40,40,Sherry Decker,['University of California Berkeley'],Marketing,[2011],94109,Banking,Product Manager,Japan,['Japanese' 'Chinese'],[ 73 84 4 1 21 111 56 65 73 97],['tag_17']


# Combine subgroups

Combine subgroups into a  4/5/6/7/8 ppl groups

In [9]:
# # helper functions, loaded to .py

# def subgroup_compare(subgroup_1, subgroup_2):
#     pair = dict()
    
#     pair['subgroup_keys'] = set().union(subgroup_1[0], subgroup_2[0])
#     pair['subgroup_uids'] = set().union(subgroup_1[3], subgroup_2[3])
#     pair['each_uids'] = [subgroup_1[3], subgroup_2[3]]
#     return pair

# # pair = subgroup_compare(subgroups[0], subgroups[1])
# # pair

# def decide_merge_subgroups(data, pair, similarity_shared_by=3):
#     tmp = combined_inverted_index(data.loc[data['index'].isin(list(pair['subgroup_uids']))], ii_input)

#     # we want to merge the subgroups, each subgroup is grouped by its own reason
#     # we need to find other similarity b/w both to combine them

#     # explore other cols
#     for key in set(priority_keys) - pair['subgroup_keys']:

#         for item in tmp[key].items():
#             # A similarity is shared by 3 + ppl
#             if len(item[1]) >= similarity_shared_by and len(item[1]-pair['each_uids'][0]) != 0 and len(item[1]-pair['each_uids'][1]) != 0:
#                 print(f'Group of {len(pair["subgroup_uids"])} ppl is formed based on {key}')
#                 return True
#             else:
#                 return False

# def decide_merge_single_user(data, group_set, individual_set, similarity_cnt_threshold=2, similarity_shared_by=3):
#     # if new fd <= 2, don't merge
#     if len(group_set - (set([int(item) for item in data.iloc[individual_set].connected[1:-1].split(' ') if item != '']))) <= 2:
#         return False
    
#     similarity_count = 0
#     tmp = combined_inverted_index(data.loc[data['index'].isin(list(group_set.union({individual_set})))], ii_input)
#     for key in set(priority_keys):
        
#         for item in tmp[key].items():
#             # A similarity is shared by 3 + ppl and include single user
#             if len(item[1]) >= similarity_shared_by and len(item[1]-{individual_set}) != 0:
#                 similarity_count += 1

#     # 2 similarity
#     if similarity_count >= similarity_cnt_threshold:
#         return True
#     else:
#         return False        

# def which_group(data, priority_keys, groups, uid, similarity_cnt_threshold=2, similarity_shared_by=3):    
#     lst = list(enumerate([(decide_merge_single_user(data, group, uid, similarity_cnt_threshold, similarity_shared_by), len(group)) for group in groups]))
    
#     for i in [4, 5, 6]:
#         res = [item for item in lst if item[1][0] and item[1][1]==i]
#         if res == []:
#             pass
#         else:
#             return res[0][0]
#     return -1

In [10]:
subgroups_remain = list(range(len(subgroups)))

len(subgroups_remain)

47

In [11]:
ids_in_groups = set()
groups = list()

In [12]:
for shared_by in [3, 2]:
    print(f'similarity shared by: {shared_by}')
    count = 0
    while len(subgroups_remain) > 2 and count < 20:
        count += 1
        lst = list(combinations(subgroups_remain, 2))
        np.random.shuffle(lst)
        for subgroups_pair in lst:
            pair = subgroup_compare(subgroups[subgroups_pair[0]], subgroups[subgroups_pair[1]])
            if decide_merge_subgroups(data, ii_input, priority_keys, pair, similarity_shared_by=3):
                ids_in_groups.update(pair['subgroup_uids'])
                groups.append(pair['subgroup_uids'])
                subgroups_remain = [ele for ele in subgroups_remain if ele not in list(subgroups_pair)]
                break

    print(f'\nRemain users: {len(remain_users(data, ids_in_groups).index)}\n')


# count = 0
# while len(subgroups_remain) > 2 and count < 20:
#     count += 1
#     lst = list(combinations(subgroups_remain, 2))
#     np.random.shuffle(lst)
#     for subgroups_pair in lst:
#         pair = subgroup_compare(subgroups[subgroups_pair[0]], subgroups[subgroups_pair[1]])
#         if decide_merge_subgroups(data, pair, similarity_shared_by=2):
#             ids_in_groups.update(pair['subgroup_uids'])
#             groups.append(pair['subgroup_uids'])
#             subgroups_remain = [ele for ele in subgroups_remain if ele not in list(subgroups_pair)]
#             break

# print(f'\nRemain users: {len(remain_users(data, ids_in_groups).index)}')

similarity shared by: 3
Group of 4 ppl is formed based on language
Group of 6 ppl is formed based on language
Group of 6 ppl is formed based on language
Group of 4 ppl is formed based on language
Group of 5 ppl is formed based on school
Group of 5 ppl is formed based on school
Group of 4 ppl is formed based on priority_tags
Group of 4 ppl is formed based on school
Group of 6 ppl is formed based on school
Group of 5 ppl is formed based on language
Group of 6 ppl is formed based on school
Group of 5 ppl is formed based on language
Group of 5 ppl is formed based on language
Group of 4 ppl is formed based on language
Group of 5 ppl is formed based on language
Group of 5 ppl is formed based on title
Group of 4 ppl is formed based on priority_tags
Group of 4 ppl is formed based on school
Group of 7 ppl is formed based on school

Remain users: 26

similarity shared by: 2

Remain users: 26



# Assign the remained individuals

Assign them to the existed groups

In [13]:
remain_users(data, ids_in_groups).index

# len(ids_in_groups)

Int64Index([  9,  10,  17,  22,  26,  33,  40,  47,  52,  54,  65,  69,  70,
             77,  83,  85,  90,  91,  97,  99, 101, 103, 105, 108, 117, 118],
           dtype='int64')

In [15]:
for uid in remain_users(data, ids_in_groups).index:
    
    group_idx = which_group(data, priority_keys, ii_input, groups, uid, similarity_cnt_threshold=1, similarity_shared_by=2)
    
    if group_idx == -1:
        print(f'{uid} cannot find subgroup')
    else:
        groups[group_idx].update({uid})
        ids_in_groups.update({uid})
        print(f'{uid} is assigned to {groups[group_idx]}')
    
    
#     print(which_group(data, priority_keys, groups, uid))

9 is assigned to {84, 9, 73, 30, 57}
10 is assigned to {51, 56, 25, 10, 12}
17 is assigned to {80, 17, 98, 37, 93}
22 is assigned to {68, 22, 45, 46, 63}
26 is assigned to {112, 113, 81, 49, 26}
33 is assigned to {33, 2, 19, 43, 28}
40 is assigned to {110, 40, 72, 42, 78}
47 is assigned to {84, 9, 73, 47, 30, 57}
52 is assigned to {51, 52, 56, 25, 10, 12}
54 is assigned to {1, 18, 82, 54, 27, 79}
65 is assigned to {96, 65, 66, 35, 88, 109}
69 is assigned to {80, 17, 98, 37, 69, 93}
70 is assigned to {68, 22, 70, 45, 46, 63}
77 is assigned to {16, 114, 67, 53, 77, 94}
83 is assigned to {50, 83, 36, 21, 119, 60}
85 is assigned to {32, 115, 85, 89, 44, 76}
90 is assigned to {112, 113, 81, 49, 90, 26}
91 is assigned to {116, 14, 55, 74, 91, 62}
97 is assigned to {0, 97, 58, 6, 104, 106}
99 is assigned to {33, 2, 19, 99, 43, 28}
101 is assigned to {110, 40, 101, 72, 42, 78}
103 is assigned to {84, 103, 9, 73, 47, 30, 57}
105 is assigned to {64, 34, 4, 100, 20, 105, 107}
108 is assigned to {

In [16]:
groups

[{9, 30, 47, 57, 73, 84, 103},
 {4, 20, 34, 64, 100, 105, 107},
 {3, 5, 11, 29, 39, 59, 108},
 {10, 12, 25, 51, 52, 56, 117},
 {1, 18, 27, 54, 79, 82, 118},
 {35, 65, 66, 88, 96, 109},
 {17, 37, 69, 80, 93, 98},
 {22, 45, 46, 63, 68, 70},
 {8, 13, 23, 31, 71, 75},
 {16, 53, 67, 77, 94, 114},
 {7, 41, 48, 86, 87, 95},
 {21, 36, 50, 60, 83, 119},
 {32, 44, 76, 85, 89, 115},
 {26, 49, 81, 90, 112, 113},
 {14, 55, 62, 74, 91, 116},
 {0, 6, 58, 97, 104, 106},
 {2, 19, 28, 33, 43, 99},
 {40, 42, 72, 78, 101, 110},
 {15, 24, 38, 61, 92, 102, 111}]